In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle
import os 
import numpy as np
import getpass
import os


input_test_file = "/home/jeans/nvaitc/battery_timeseries/resources/raw/test_bat_data.pkl"

with open(input_test_file, "rb") as file:
    test_dict = pickle.load(file)
    
for battery_id in test_dict:
    trimmed_arr = np.trim_zeros(test_dict[battery_id]['q_d_n'], 'b')
    test_dict[battery_id]['trimmed_q_d_n'] = list(trimmed_arr)
    
last_k_th_cycles_list = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# calculate slope from first cycle to the last cycle
for battery_id in test_dict:
    q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
    cycle = test_dict[battery_id]['cycle']
    slope = (q_d_n_values[-1] - q_d_n_values[0]) / cycle
    test_dict[battery_id]['slope_all_cycles'] = slope
    
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = (q_d_n_values[-1] - q_d_n_values[-k]) / k
        else:
            slope = np.nan
        test_dict[battery_id][f'slope_last_{k}_cycles'] = slope
        
#calculate a mean gradient of the last k cycles using numpy gradient
for k in last_k_th_cycles_list:
    for battery_id in test_dict:
        q_d_n_values  = test_dict[battery_id]['trimmed_q_d_n']
        cycle = test_dict[battery_id]['cycle']
        if cycle > k:
            slope = np.gradient(q_d_n_values[-k:], 1)
            mean_slope = np.mean(slope)
        else:
            mean_slope = np.nan
        test_dict[battery_id][f'mean_grad_last_{k}_cycles'] = mean_slope

In [3]:
sample_battery_id = 'b3c45'
print("battery_id:", test_dict.keys())
features = test_dict[sample_battery_id].keys()
print(f"sample_battery_id ({sample_battery_id}) features:")
for feature in features:
    print(f'    -{feature} : ({type(test_dict[sample_battery_id][feature])})')

battery_id: dict_keys(['b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30', 'b3c31', 'b3c33', 'b3c34', 'b3c35', 'b3c36', 'b3c40', 'b3c41', 'b3c42', 'b3c43', 'b3c44', 'b3c45'])
sample_battery_id (b3c45) features:
    -q_d_n : (<class 'list'>)
    -cycle : (<class 'int'>)
    -trimmed_q_d_n : (<class 'list'>)
    -slope_all_cycles : (<class 'float'>)
    -slope_last_10_cycles : (<class 'float'>)
    -slope_last_50_cycles : (<class 'float'>)
    -slope_last_100_cycles : (<class 'float'>)
    -slope_last_200_cycles : (<class 'float'>)
    -slope_last_300_cycles : (<class 'float'>)
    -slope_last_400_cycles : (<class 'float'>)
    -slope_last_500_cycles : (<class 'float'>)
    -slope_last_600_cycles : (<class 'float'>)
    -slope_last_700_cycles : (<class 'float'>)
    -slope_last_800_cycles : 

In [4]:
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)

graph.refresh_schema()
print(graph.schema)

/tmp/ipykernel_101141/636819369.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(refresh_schema=False)


Node properties:
ChargingPolicy {charging_policy: STRING}
Battery {battery_id: STRING, total_cycles: INTEGER, slope_all_cycles: FLOAT, trimmed_q_d_n_avg: FLOAT, name: STRING, slope_last_10_cycles: FLOAT, slope_last_50_cycles: FLOAT, slope_last_100_cycles: FLOAT, slope_last_200_cycles: FLOAT, slope_last_300_cycles: FLOAT, slope_last_400_cycles: FLOAT, slope_last_500_cycles: FLOAT, slope_last_600_cycles: FLOAT, slope_last_700_cycles: FLOAT, slope_last_800_cycles: FLOAT, slope_last_900_cycles: FLOAT, slope_last_1000_cycles: FLOAT, mean_grad_last_10_cycles: FLOAT, mean_grad_last_50_cycles: FLOAT, mean_grad_last_100_cycles: FLOAT, mean_grad_last_200_cycles: FLOAT, mean_grad_last_300_cycles: FLOAT, mean_grad_last_400_cycles: FLOAT, mean_grad_last_500_cycles: FLOAT, mean_grad_last_600_cycles: FLOAT, mean_grad_last_700_cycles: FLOAT, mean_grad_last_800_cycles: FLOAT, mean_grad_last_900_cycles: FLOAT, mean_grad_last_1000_cycles: FLOAT}
Relationship properties:

The relationships:
(:ChargingPoli

## Example Query

In [8]:
test_key = 'b3c0'
test_battery = test_dict[test_key]
test_battery.keys()

feature = "slope_last_500_cycles"
test_slope = test_battery[f"{feature}"]  # Change this to compare different features

# Define similarity threshold and number of results
THRESHOLD = 10  # Adjust based on desired similarity
TOP_K = 3  # Number of closest matches to return

# Query Neo4j for batteries within the similarity threshold & fetch charging policy
query = f"""
MATCH (b:Battery)
WHERE abs(b.{feature} - {test_slope}) < {THRESHOLD}
RETURN b.battery_id AS battery_id, 
       b.{feature} AS {feature}, 
       abs(b.{feature} - {test_slope}) AS similarity
ORDER BY similarity ASC
LIMIT {TOP_K}
"""

# Execute query
neo4j_results = graph.query(query)
print(neo4j_results)
# Print results
print(f"Test Battery: {test_key} ({feature} = {test_slope})")
print(f"Closest {TOP_K} Matches in Neo4j (within threshold {THRESHOLD}):")
for idx, battery in enumerate(neo4j_results):
    print(f"{idx+1}. Battery: {battery['battery_id']} - Slope: {battery[f'{feature}']} (Diff: {battery['similarity']})")
    print(f"   Charging Policy: {battery['charging_policy']}\n")

[{'battery_id': 'b1c11', 'slope_last_500_cycles': -0.00034225058555603026, 'similarity': 2.099394798278794e-06}, {'battery_id': 'b1c18', 'slope_last_500_cycles': -0.00034250414371490476, 'similarity': 2.3529529571533e-06}, {'battery_id': 'b1c5', 'slope_last_500_cycles': -0.0003480168581008911, 'similarity': 7.865667343139645e-06}]
Test Battery: b3c0 (slope_last_500_cycles = -0.00034015119075775146)
Closest 3 Matches in Neo4j (within threshold 10):
1. Battery: b1c11 - Slope: -0.00034225058555603026 (Diff: 2.099394798278794e-06)


KeyError: 'charging_policy'

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
if os.environ.get("NVIDIA_API_KEY") is None:
    os.environ["NVIDIA_API_KEY"] = input()

# Set up NVIDIA ChatNVIDIA LLM
llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
)

# Query the graph database
query = """
MATCH (b:Battery)-[:HAS_CYCLE]->(c:Cycle)
RETURN b.batteryID AS batteryID, c.cycle_number AS cycle, c.discharge_capacity AS capacity
LIMIT 10
"""
# result = graph.query(query)

# Format results for the LLM
data = [{"batteryID": record["batteryID"], "cycle": record["cycle"], "capacity": record["capacity"]} for record in result]

# Pass to LLM for further analysis
prompt = f'''Format this text for presentation slide and correct my grammar:\n\nbattery_dataset : dict() with batteryID as key
Example 
->battery_dataset['b1c1'] : dict with 6 keys ['q_d_n', 'trimmed_q_d_n', 'cycle', 'mean_backhalf_grad', 'gradient_q_d_n']

Attributes
- cycle : int 
- q_d_n : array of float showing the discharge capacity left? of each cycle. (unused)
- trimmed_q_d_n : q_d_n but trimmed 0s off.
- gradient_q_d_n : array of float showing the grad between discharge capacity left (length == cycle).
- mean_backhalf_grad : floating point number representing the mean of gradient from all of the backhalf element of the array in q_d_n`
-charging_policy : str'''
print(prompt)
for chunk in llm.stream([{"role": "user", "content": prompt}]):
    print(chunk.content, end="")

/home/jeans/miniconda3/envs/tserie/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_common.py:176: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


NameError: name 'result' is not defined